In [ ]:
from fractal_zero.trainers.muzero_discriminator import FractalMuZeroDiscriminatorTrainer, FMZGModel
from fractal_zero.models.joint_model import JointModel

import torch

In [ ]:
from fractal_zero.data.expert_dataset import ExpertDatasetGenerator
from fractal_zero.vectorized_environment import load_environment

env = load_environment("CartPole-v0")


rep_model = torch.nn.Identity()

dyn_model = torch.nn.Sequential(
    torch.nn.Linear(5, 4),  # obs space + action embedding = 4 + 1 = 5
    torch.nn.ReLU(),
    torch.nn.Linear(4, 4),
)
disc_model = torch.nn.Sequential(
    torch.nn.Linear(5, 1),
    torch.nn.Sigmoid(),  # between 0 and 1 confidence values.
)

action_vec = lambda x: torch.tensor(x).flatten().int().item()

model = FMZGModel(
    rep_model, 
    dyn_model, 
    disc_model, 
    num_walkers=2, 
    action_vectorizer=action_vec,
)

expert_policy_model = lambda x: env.action_space.sample()  # TODO: use an actual model...
expert_dataset = ExpertDatasetGenerator(
    expert_policy_model,
    env,
    action_vectorizer=action_vec,
)

discriminator_optimizer = torch.optim.SGD([*rep_model.parameters(), *disc_model.parameters()], lr=0.01)

trainer = FractalMuZeroDiscriminatorTrainer(env, model, expert_dataset, discriminator_optimizer)

In [ ]:
for _ in range(100):
    print(trainer.train_step(8))